In [1]:
num_samples = 3
interval = 33.0e-3
num_frames = 100
min_sigma = 1
max_sigma = 4
threshold = 50.0
overlap = 0.5

In [2]:
nproc = 20

In [3]:
import numpy
timepoints = numpy.linspace(0, interval * num_frames, num_frames + 1)

In [4]:
import pathlib
inputpath = pathlib.Path("./artifacts")
artifacts = pathlib.Path("./artifacts")
artifacts.mkdir(parents=True, exist_ok=True)

In [5]:
import scopyon

In [6]:
import warnings
warnings.simplefilter('ignore', RuntimeWarning)

for i in range(num_samples):
    imgs = [scopyon.Image(data) for data in numpy.load(inputpath / f"images{i:03d}.npy")]
    spots = [
        scopyon.analysis.spot_detection(
            img.as_array(), processes=nproc,
            min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold, overlap=overlap)
        for img in imgs]

    spots_ = []
    for t, data in zip(timepoints, spots):
        spots_.extend(([t] + list(row) for row in data))
    spots_ = numpy.array(spots_)
    numpy.save(artifacts / f"spots{i:03d}.npy", spots_)
    
    print("{} spots are detected in {} frames.".format(len(spots_), len(imgs)))

warnings.resetwarnings()

19602 spots are detected in 100 frames.
19125 spots are detected in 100 frames.
19068 spots are detected in 100 frames.


In [7]:
!ls ./artifacts

config.yaml    images001.npy  inputs000.npy  inputs002.npy  spots001.npy
images000.npy  images002.npy  inputs001.npy  spots000.npy   spots002.npy
